In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0
y_train = y_train[:,0].astype(np.int32)
y_test = y_test[:,0].astype(np.int32)

#Creating validation sets
val_size = 2000
X_valid, X_train = X_train[:val_size], X_train[val_size:]
y_valid, y_train = y_train[:val_size], y_train[val_size:]

c:\users\janin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\janin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\janin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\janin\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_hidden1 = 4096
n_hidden2 = 4096
n_outputs = len(np.unique(y_train))

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z

############

conv1_fmaps = 96
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 128
conv2_ksize = 5
conv2_stride = 1
conv2_pad = "SAME"

conv3_fmaps = 256
conv3_ksize = 5
conv3_stride = 1
conv3_pad = "SAME"

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="X")
    y = tf.placeholder(tf.int64, shape=(None), name="y")

conv1 = tf.layers.conv2d(X, filters=conv1_fmaps, kernel_size=conv1_ksize, 
                         strides=conv1_stride, padding=conv1_pad, name="conv1")  
bn1 = tf.layers.batch_normalization(conv1, training=True, momentum=0.9)
bn1_act = tf.nn.relu(bn1)

pool1 = tf.nn.max_pool(bn1_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad, name="conv2")   
bn2 = tf.layers.batch_normalization(conv2, training=True, momentum=0.9)
bn2_act = tf.nn.relu(bn2)

pool2 = tf.nn.max_pool(bn2_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad, name="conv3")
bn3 = tf.layers.batch_normalization(conv3, training=True, momentum=0.9)
bn3_act = tf.nn.relu(bn3)

pool3 = tf.nn.max_pool(bn3_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

n_fc1 = np.int64(pool3.shape[1]*pool3.shape[2]*pool3.shape[3])
pool3_flat = tf.reshape(pool3, shape=[-1, n_fc1])

###################

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(pool3_flat, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")

def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        a = 3.7
        lambdaVal = tf.constant(lambdaValScalar, W.dtype)
        aLambda = tf.scalar_mul(a, lambdaVal)
        condMat = tf.multiply(tf.cast(tf.abs(W) > lambdaVal, W.dtype), tf.cast(tf.abs(W) <= aLambda, W.dtype))
        lambdaValSq = tf.pow(lambdaVal, 2)
        absW = tf.abs(W)
        reg_value = tf.cast(absW <= lambdaVal, W.dtype)*tf.scalar_mul(lambdaVal, absW) + \
                    condMat*tf.scalar_mul(-0.5/(a-1), tf.pow(W, 2) - 2*tf.scalar_mul(aLambda, absW) + lambdaValSq) + \
                    tf.cast(absW > aLambda, W.dtype)*tf.scalar_mul(0.5*(a+1), lambdaValSq)
        return tf.reduce_sum(reg_value)
    elif type == 9:
        b = 2
        lambdaVal = tf.constant(lambdaValScalar, W.dtype)
        absW = tf.abs(W)
        reg_value = tf.cast(absW > b*lambdaVal, W.dtype)*tf.sign(W)*(0.5*b*tf.pow(lambdaVal, 2)) + \
                    tf.cast(absW <= b*lambdaVal, W.dtype)*tf.sign(W)*tf.scalar_mul(lambdaVal, absW - \
                                                                       tf.scalar_mul(1/(2*b*lambdaVal), tf.pow(W, 2))) 
        return tf.reduce_sum(reg_value)
    elif type == 10:
        gammaVal = pow(10.0, 2)
        #gammaVal = tf.sqrt(tf.abs(tf.div(tf.reduce_sum(W), tf.reduce_sum(tf.pow(W, 3)))))
        reg_value =  (2/math.pi)*tf.atan(tf.scalar_mul(gammaVal, tf.abs(W)))
        return tf.reduce_sum(reg_value)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 10
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
                                  
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    

    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices,:,:,:], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, acc_val = sess.run([loss_total, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        
        print("{}\tValidation losses: {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20200208153222.ckpt") # or better, use save_path
    n_batches = len(X_test) // 100
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        #print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [ ]:
    for i in [1, 2, 3]:
        for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
            print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j, "\tGAMMA:  ", 100)
            execute_code(rand_seed=i, lambda_exp=j)


RANDOM SEED:   1 	LAMBDA_EXP:   -4.0 	GAMMA:   100
Now: 20200406213102
0	Validation losses: 341.2034	Best Loss: 341.2034 (0)	Accuracy: 62.15%
1	Validation losses: 72.8612	Best Loss: 72.8612 (0)	Accuracy: 58.70%
2	Validation losses: 31.8035	Best Loss: 31.8035 (0)	Accuracy: 63.20%
3	Validation losses: 30.1207	Best Loss: 30.1207 (0)	Accuracy: 71.20%
4	Validation losses: 35.6772	Best Loss: 30.1207 (1)	Accuracy: 75.30%
5	Validation losses: 42.1049	Best Loss: 30.1207 (2)	Accuracy: 76.00%
6	Validation losses: 47.8114	Best Loss: 30.1207 (3)	Accuracy: 79.20%
7	Validation losses: 52.4128	Best Loss: 30.1207 (4)	Accuracy: 76.10%
8	Validation losses: 57.2901	Best Loss: 30.1207 (5)	Accuracy: 80.65%
9	Validation losses: 62.4557	Best Loss: 30.1207 (6)	Accuracy: 80.10%
10	Validation losses: 64.2699	Best Loss: 30.1207 (7)	Accuracy: 81.50%
11	Validation losses: 71.7773	Best Loss: 30.1207 (8)	Accuracy: 81.05%
12	Validation losses: 79.3717	Best Loss: 30.1207 (9)	Accuracy: 80.15%
13	Validation losses: 85.96

33	Validation losses: 2.5974	Best Loss: 2.5974 (0)	Accuracy: 86.05%
34	Validation losses: 2.4435	Best Loss: 2.4435 (0)	Accuracy: 86.00%
35	Validation losses: 2.1974	Best Loss: 2.1974 (0)	Accuracy: 85.95%
36	Validation losses: 2.0743	Best Loss: 2.0743 (0)	Accuracy: 85.75%
37	Validation losses: 1.8360	Best Loss: 1.8360 (0)	Accuracy: 85.80%
38	Validation losses: 1.6894	Best Loss: 1.6894 (0)	Accuracy: 85.75%
39	Validation losses: 1.4477	Best Loss: 1.4477 (0)	Accuracy: 86.10%
40	Validation losses: 1.3024	Best Loss: 1.3024 (0)	Accuracy: 86.00%
41	Validation losses: 1.3122	Best Loss: 1.3024 (1)	Accuracy: 85.90%
42	Validation losses: 1.3243	Best Loss: 1.3024 (2)	Accuracy: 85.95%
43	Validation losses: 1.3214	Best Loss: 1.3024 (3)	Accuracy: 86.00%
44	Validation losses: 1.3281	Best Loss: 1.3024 (4)	Accuracy: 85.90%
45	Validation losses: 1.3124	Best Loss: 1.3024 (5)	Accuracy: 86.05%
46	Validation losses: 1.2917	Best Loss: 1.2917 (0)	Accuracy: 85.65%
47	Validation losses: 1.3018	Best Loss: 1.2917 (

19	Validation losses: 2.4100	Best Loss: 2.4100 (0)	Accuracy: 82.35%
20	Validation losses: 2.9368	Best Loss: 2.4100 (1)	Accuracy: 77.50%
21	Validation losses: 2.5572	Best Loss: 2.4100 (2)	Accuracy: 80.70%
22	Validation losses: 2.5033	Best Loss: 2.4100 (3)	Accuracy: 80.75%
23	Validation losses: 2.0725	Best Loss: 2.0725 (0)	Accuracy: 82.05%
24	Validation losses: 2.1646	Best Loss: 2.0725 (1)	Accuracy: 82.65%
25	Validation losses: 2.1753	Best Loss: 2.0725 (2)	Accuracy: 83.20%
26	Validation losses: 1.9171	Best Loss: 1.9171 (0)	Accuracy: 84.40%
27	Validation losses: 1.7251	Best Loss: 1.7251 (0)	Accuracy: 84.10%
28	Validation losses: 1.9664	Best Loss: 1.7251 (1)	Accuracy: 84.10%
29	Validation losses: 1.9425	Best Loss: 1.7251 (2)	Accuracy: 82.80%
30	Validation losses: 1.6973	Best Loss: 1.6973 (0)	Accuracy: 85.55%
31	Validation losses: 1.6565	Best Loss: 1.6565 (0)	Accuracy: 85.40%
32	Validation losses: 1.6361	Best Loss: 1.6361 (0)	Accuracy: 85.90%
33	Validation losses: 1.5563	Best Loss: 1.5563 (

11	Validation losses: 3.6182	Best Loss: 3.6182 (0)	Accuracy: 81.85%
12	Validation losses: 3.0159	Best Loss: 3.0159 (0)	Accuracy: 82.60%
13	Validation losses: 2.8084	Best Loss: 2.8084 (0)	Accuracy: 83.00%
14	Validation losses: 2.6247	Best Loss: 2.6247 (0)	Accuracy: 81.50%
15	Validation losses: 2.2079	Best Loss: 2.2079 (0)	Accuracy: 83.30%
16	Validation losses: 2.2388	Best Loss: 2.2079 (1)	Accuracy: 82.30%
17	Validation losses: 2.5007	Best Loss: 2.2079 (2)	Accuracy: 78.30%
18	Validation losses: 2.2762	Best Loss: 2.2079 (3)	Accuracy: 81.10%
19	Validation losses: 2.2444	Best Loss: 2.2079 (4)	Accuracy: 80.70%
20	Validation losses: 2.3037	Best Loss: 2.2079 (5)	Accuracy: 78.10%
21	Validation losses: 1.9522	Best Loss: 1.9522 (0)	Accuracy: 82.60%
22	Validation losses: 1.8518	Best Loss: 1.8518 (0)	Accuracy: 83.80%
23	Validation losses: 2.1174	Best Loss: 1.8518 (1)	Accuracy: 81.95%
24	Validation losses: 2.1098	Best Loss: 1.8518 (2)	Accuracy: 81.65%
25	Validation losses: 2.0353	Best Loss: 1.8518 (

67	Validation losses: 1.3521	Best Loss: 1.3040 (1)	Accuracy: 85.20%
68	Validation losses: 1.3032	Best Loss: 1.3032 (0)	Accuracy: 85.50%
69	Validation losses: 1.2987	Best Loss: 1.2987 (0)	Accuracy: 85.55%
70	Validation losses: 1.2950	Best Loss: 1.2950 (0)	Accuracy: 85.75%
71	Validation losses: 1.2923	Best Loss: 1.2923 (0)	Accuracy: 85.85%
72	Validation losses: 1.2878	Best Loss: 1.2878 (0)	Accuracy: 86.05%
73	Validation losses: 1.2829	Best Loss: 1.2829 (0)	Accuracy: 86.15%
74	Validation losses: 1.2821	Best Loss: 1.2821 (0)	Accuracy: 86.05%
75	Validation losses: 1.2796	Best Loss: 1.2796 (0)	Accuracy: 86.00%
76	Validation losses: 1.2731	Best Loss: 1.2731 (0)	Accuracy: 86.10%
77	Validation losses: 1.2735	Best Loss: 1.2731 (1)	Accuracy: 86.05%
78	Validation losses: 1.2702	Best Loss: 1.2702 (0)	Accuracy: 86.00%
79	Validation losses: 1.2687	Best Loss: 1.2687 (0)	Accuracy: 85.95%
80	Validation losses: 1.2680	Best Loss: 1.2680 (0)	Accuracy: 85.95%
81	Validation losses: 1.2680	Best Loss: 1.2680 (

187	Validation losses: 1.2109	Best Loss: 1.2109 (0)	Accuracy: 85.95%
188	Validation losses: 1.2107	Best Loss: 1.2107 (0)	Accuracy: 86.00%
189	Validation losses: 1.2107	Best Loss: 1.2107 (0)	Accuracy: 86.05%
190	Validation losses: 1.2100	Best Loss: 1.2100 (0)	Accuracy: 86.10%
191	Validation losses: 1.2097	Best Loss: 1.2097 (0)	Accuracy: 86.00%
192	Validation losses: 1.2086	Best Loss: 1.2086 (0)	Accuracy: 86.15%
193	Validation losses: 1.2078	Best Loss: 1.2078 (0)	Accuracy: 86.10%
194	Validation losses: 1.2079	Best Loss: 1.2078 (1)	Accuracy: 86.10%
195	Validation losses: 1.2095	Best Loss: 1.2078 (2)	Accuracy: 86.20%
196	Validation losses: 1.2092	Best Loss: 1.2078 (3)	Accuracy: 86.15%
197	Validation losses: 1.2073	Best Loss: 1.2073 (0)	Accuracy: 86.05%
198	Validation losses: 1.2072	Best Loss: 1.2072 (0)	Accuracy: 86.00%
199	Validation losses: 1.2071	Best Loss: 1.2071 (0)	Accuracy: 86.00%
200	Validation losses: 1.2079	Best Loss: 1.2071 (1)	Accuracy: 86.00%
201	Validation losses: 1.2073	Best

54	Validation losses: 1.4034	Best Loss: 1.4034 (0)	Accuracy: 87.00%
55	Validation losses: 1.4000	Best Loss: 1.4000 (0)	Accuracy: 86.80%
56	Validation losses: 1.3911	Best Loss: 1.3911 (0)	Accuracy: 87.05%
57	Validation losses: 1.3841	Best Loss: 1.3841 (0)	Accuracy: 87.20%
58	Validation losses: 1.3777	Best Loss: 1.3777 (0)	Accuracy: 86.95%
59	Validation losses: 1.3737	Best Loss: 1.3737 (0)	Accuracy: 86.70%
60	Validation losses: 1.3709	Best Loss: 1.3709 (0)	Accuracy: 86.70%
61	Validation losses: 1.3728	Best Loss: 1.3709 (1)	Accuracy: 87.20%
62	Validation losses: 1.3676	Best Loss: 1.3676 (0)	Accuracy: 87.30%
63	Validation losses: 1.3625	Best Loss: 1.3625 (0)	Accuracy: 87.10%
64	Validation losses: 1.3565	Best Loss: 1.3565 (0)	Accuracy: 87.00%
65	Validation losses: 1.3471	Best Loss: 1.3471 (0)	Accuracy: 87.35%
66	Validation losses: 1.3379	Best Loss: 1.3379 (0)	Accuracy: 87.10%
67	Validation losses: 1.3338	Best Loss: 1.3338 (0)	Accuracy: 87.05%
68	Validation losses: 1.3276	Best Loss: 1.3276 (

174	Validation losses: 1.2292	Best Loss: 1.2292 (0)	Accuracy: 86.75%
175	Validation losses: 1.2289	Best Loss: 1.2289 (0)	Accuracy: 86.75%
176	Validation losses: 1.2288	Best Loss: 1.2288 (0)	Accuracy: 86.80%
177	Validation losses: 1.2290	Best Loss: 1.2288 (1)	Accuracy: 86.75%
178	Validation losses: 1.2292	Best Loss: 1.2288 (2)	Accuracy: 86.90%
179	Validation losses: 1.2294	Best Loss: 1.2288 (3)	Accuracy: 86.85%
180	Validation losses: 1.2287	Best Loss: 1.2287 (0)	Accuracy: 86.90%
181	Validation losses: 1.2278	Best Loss: 1.2278 (0)	Accuracy: 86.85%
182	Validation losses: 1.2276	Best Loss: 1.2276 (0)	Accuracy: 86.85%
183	Validation losses: 1.2270	Best Loss: 1.2270 (0)	Accuracy: 86.80%
184	Validation losses: 1.2270	Best Loss: 1.2270 (1)	Accuracy: 86.80%
185	Validation losses: 1.2278	Best Loss: 1.2270 (2)	Accuracy: 86.70%
186	Validation losses: 1.2274	Best Loss: 1.2270 (3)	Accuracy: 86.75%
187	Validation losses: 1.2267	Best Loss: 1.2267 (0)	Accuracy: 86.80%
188	Validation losses: 1.2251	Best

41	Validation losses: 1.4647	Best Loss: 1.4647 (0)	Accuracy: 86.55%
42	Validation losses: 1.4628	Best Loss: 1.4628 (0)	Accuracy: 86.55%
43	Validation losses: 1.4609	Best Loss: 1.4609 (0)	Accuracy: 86.55%
44	Validation losses: 1.4560	Best Loss: 1.4560 (0)	Accuracy: 86.50%
45	Validation losses: 1.4526	Best Loss: 1.4526 (0)	Accuracy: 86.50%
46	Validation losses: 1.4480	Best Loss: 1.4480 (0)	Accuracy: 86.60%
47	Validation losses: 1.4454	Best Loss: 1.4454 (0)	Accuracy: 86.45%
48	Validation losses: 1.4363	Best Loss: 1.4363 (0)	Accuracy: 86.40%
49	Validation losses: 1.4310	Best Loss: 1.4310 (0)	Accuracy: 86.40%
50	Validation losses: 1.4225	Best Loss: 1.4225 (0)	Accuracy: 86.60%
51	Validation losses: 1.4163	Best Loss: 1.4163 (0)	Accuracy: 86.65%
52	Validation losses: 1.4087	Best Loss: 1.4087 (0)	Accuracy: 86.60%
53	Validation losses: 1.3976	Best Loss: 1.3976 (0)	Accuracy: 87.05%
54	Validation losses: 1.3898	Best Loss: 1.3898 (0)	Accuracy: 86.80%
55	Validation losses: 1.3813	Best Loss: 1.3813 (

161	Validation losses: 1.1697	Best Loss: 1.1697 (0)	Accuracy: 86.70%
162	Validation losses: 1.1695	Best Loss: 1.1695 (0)	Accuracy: 86.80%
163	Validation losses: 1.1690	Best Loss: 1.1690 (0)	Accuracy: 86.80%
164	Validation losses: 1.1689	Best Loss: 1.1689 (0)	Accuracy: 86.80%
165	Validation losses: 1.1689	Best Loss: 1.1689 (1)	Accuracy: 86.75%
166	Validation losses: 1.1686	Best Loss: 1.1686 (0)	Accuracy: 86.75%
167	Validation losses: 1.1680	Best Loss: 1.1680 (0)	Accuracy: 86.75%
168	Validation losses: 1.1673	Best Loss: 1.1673 (0)	Accuracy: 86.85%
169	Validation losses: 1.1678	Best Loss: 1.1673 (1)	Accuracy: 86.80%
170	Validation losses: 1.1681	Best Loss: 1.1673 (2)	Accuracy: 86.75%
171	Validation losses: 1.1680	Best Loss: 1.1673 (3)	Accuracy: 86.75%
172	Validation losses: 1.1673	Best Loss: 1.1673 (0)	Accuracy: 86.80%
173	Validation losses: 1.1665	Best Loss: 1.1665 (0)	Accuracy: 86.80%
174	Validation losses: 1.1659	Best Loss: 1.1659 (0)	Accuracy: 86.85%
175	Validation losses: 1.1652	Best

In [ ]:
 
    -